<a href="https://colab.research.google.com/github/Anastas1aMakarova/PythonTest/blob/main/PyTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd #для работы с табличными данными
from datetime import datetime #для работы с датами и временем

#функция для загрузки данных
def extract_data():
    data = {
        #загрузка данных из Excel
        'airports': pd.read_excel('data/airports.xlsx'),
        'aircrafts': pd.read_excel('data/aircrafts.xlsx'),
        'tickets': pd.read_excel('data/tickets.xlsx'),
        'flights': pd.read_excel('data/flights.xlsx'),
        'ticket_flights': pd.read_excel('data/ticket_flights.xlsx'),
        #загрузка данных из HTML-файла - обращаемся к первому объекту списка
        'festivals': pd.read_html('festivals/Festival News.htm')[0]
    }
    return data

#функция для преобразования данных
def transform_data(data):
    #подготовка таблицы с рейсами
    flights = data['flights']
    #преобразование даты рейса в неделю и вынесение в отдельный столбец
    flights['week'] = pd.to_datetime(flights['departure_time']).dt.isocalendar().week
    #объединение с таблицей по самолетам  и таблицей по аэропортам (наименования столбцов с одними и теми же данными разные)
    flights_merged = flights.merge(data['aircrafts'], on='aircraft_code').merge(data['airports'], left_on='departure_airport', right_on='airport_code')

    #подготовка таблицы с билетами (совпадений по рейсам в таблице ticket_flights и таблице flights нет, поэтому результат объединения - пустая таблица)
    ticket_merged = data['ticket_flights'].merge(flights, on='flight_id').merge(data['airports'], left_on='departure_airport', right_on='airport_code')

    #подготовка таблицы с данными фестивалей
    festivals = data['festivals']
    #преобразование даты фестиваля в неделю и вынесение в отдельный столбец
    festivals['week'] = pd.to_datetime(festivals['Дата проведения'], format='%d.%m.%Y').dt.isocalendar().week
    #переименовываем столбцы, чтобы в дальнейшем было проще объядинить в результатную таблицу
    festivals = festivals.rename(columns={'Название фестиваля': 'festival_name',
                                          'Место проведения': 'city',
                                          'Дата проведения': 'festival_date'})

    #подсчет
    #количество вылетов из каждого аэропорта по каждой модели самолета (подсчет кол-ва элементов через size())
    flights_departure = flights_merged.groupby(['week', 'departure_airport', 'city', 'model']).size().reset_index(name='cnt_flights_departure')
    #количество прилетов из каждого аэропорта по каждой модели самолета
    flights_arrival = flights_merged.groupby(['week', 'arrival_airport', 'city', 'model']).size().reset_index(name="cnt_flights_arrival")
    #среднее количество прибывающих рейсов для каждого города (сначала кол-во по неделям и городам, затем по данным расчетам через mean() среднеее по городам)
    #поскольку считается среднее по городам, рассчитанный показатель по каждому городу дублируется в таблице напротив соответствующего города)
    avg_flights = flights_merged.groupby(['week', 'city']).size().groupby(['city']).mean().reset_index(name="average_flights")
    #количество купленных билетов для каждого города и аэропорта
    tickets_count = ticket_merged.groupby(["week", "city", "departure_airport"]).size().reset_index(name="ticket_amount")

    #объединение в результативную таблицу
    result = flights_departure.merge(
        flights_arrival, on=['week', 'city', 'model'], how='outer').merge(
        avg_flights, on=['city'], how="left").merge(
        tickets_count, on=['week', 'city', 'departure_airport'], how="left").merge(
        festivals, on=['week', 'city'], how="left")

    #удаление лишних столбцов и упорядочивание столбцов
    result = result.drop(['arrival_airport', 'festival_date'], axis=1)
    result = result.reindex(columns=[
        'week', 'city', 'departure_airport', 'model', 'festival_name',
        'cnt_flights_departure', 'cnt_flights_arrival', 'average_flights', 'ticket_amount'
    ])
    return result

#функция для выгрузки данных
def load_data(result, output_file):
    #Сохраняем в Excel
    output_file = "result.xlsx"
    result.to_excel(output_file, index=False)
    print(f"Данные сохранены в файл: {output_file}")

#ETL-процесс
def etl():
    data = extract_data()
    transformed_data = transform_data(data)
    load_data(transformed_data, "result.xlsx")

#Запуск приложения
if __name__ == "__main__":
    etl()

Данные сохранены в файл: result.xlsx
